In [ ]:
import traits.api as tr
import sympy as sp
import dill
import os
import functools
def get_dirac_delta(x, x_0=0):
    return 0
numpy_dirac =[{'DiracDelta': get_dirac_delta }, 'numpy']


CACHE_DIR = '_lambdified_cache'

def cached_lambdify(func):
    @functools.wraps(func)
    def wrapper(self):
        class_name = self.__class__.__name__
        object_name = self.name
        property_name = func.__name__
        cache_dir = CACHE_DIR
        filename = os.path.join(cache_dir, f"{class_name}_{object_name}_{property_name}.pkl")

        if not os.path.exists(cache_dir):
            os.makedirs(cache_dir)

        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                lambdified_func = dill.load(f)
        else:
            lambdified_func = func(self)
            with open(filename, 'wb') as f:
                dill.dump(lambdified_func, f)

        return lambdified_func

    return tr.cached_property(wrapper)

class GSM(tr.HasTraits):
    name = tr.Str('unnamed')

    # Example of using the new decorator
    _sig_lambdified = cached_lambdify
    def _get__sig_lambdified(self):
        return sp.lambdify((self.u_vars, self.T_var, 
                            self.Eps.as_explicit(), 
                            self.Sig.as_explicit()) + self.m_params + ('**kw',), 
                           self.sig_, numpy_dirac, cse=True)

    # Apply the new decorator to other properties
    _dDiss_dEps_lambdified = cached_lambdify
    def _get__dDiss_dEps_lambdified(self):
        return sp.lambdify((self.u_vars, self.T_var, 
                            self.Eps.as_explicit(), 
                            self.Sig.as_explicit()) + self.m_params + ('**kw',), 
                           self.dDiss_dEps_, numpy_dirac, cse=True)

    _Sig_lambdified = cached_lambdify
    def _get__Sig_lambdified(self):
        return sp.lambdify((self.u_vars, self.T_var, 
                            self.Eps.as_explicit(), 
                            self.Sig.as_explicit()) + self.m_params + ('**kw',), 
                           self.Sig_.as_explicit(), numpy_dirac, cse=True)

    _Phi_lambdified = cached_lambdify
    def _get__Phi_lambdified(self):
        return sp.lambdify((self.u_vars, self.T_var, 
                            self.Eps.as_explicit(), 
                            self.Sig.as_explicit()) + self.m_params + ('**kw',), 
                           self.Phi_, numpy_dirac, cse=True)

class A(tr.HasTraits):
    a = tr.Property()
    def _get_a(self):
        return 22


In [ ]:
gsm_x = GSM()
a_ = A()

In [ ]:
a_.a

In [ ]:
d